In [ ]:
import en_core_web_sm
nlp = en_core_web_sm.load()
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import pandas as pd
import numpy as np
from time import time
import multiprocessing
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split

#file_name = 'document_classification\Resume1.txt'
#mat=[][2]
import os
filesres = []
filesnon = []
filespred = []
pred = []
main=[]
#print(files)
for i in os.listdir("/content/sample_data/Resume"):
    if i.startswith('r'):
        filesres.append(i)

for i in os.listdir("/content/sample_data/Non resume"):
    if i.startswith('n'):
        filesnon.append(i)


for i in os.listdir("/content/sample_data/prediction"):
    if i != ".ipynb_checkpoints":
      filespred.append(i)

for file in filesres:
    x=file
    try:
        fname = open("/content/sample_data/Resume"+"/"+x).read()
    except UnicodeDecodeError:
        continue

    f1=re.sub(r'\W+', ' ',fname)

    wordnet_lemmatizer = WordNetLemmatizer()

    words = nltk.word_tokenize(f1)

    f2=" "
    for w in words:
        f2=f2+" "+wordnet_lemmatizer.lemmatize(w)

    doc = nlp(f2)

    tokens = [token.text for token in doc if not token.is_stop]

    l=[]
    l.append(tokens)
    l.append("1")
    main.append(l)


for file in filesnon:
    x=file
    try:
        fname = open("/content/sample_data/Non resume"+"/"+x).read()
    except UnicodeDecodeError:
        continue

    f1=re.sub(r'\W+', ' ',fname)
    wordnet_lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(f1)
    f2=" "
    for w in words:
        f2=f2+" "+wordnet_lemmatizer.lemmatize(w)
    doc = nlp(f2)

    tokens = [token.text for token in doc if not token.is_stop]

    l=[]
    l.append(tokens)
    l.append("0")
    main.append(l)

for file in filespred:
    x=file
    try:
        fname = open("/content/sample_data/prediction"+"/"+x).read()
    except UnicodeDecodeError:
        continue

    f1=re.sub(r'\W+', ' ',fname)
    wordnet_lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(f1)
    f2=" "
    for w in words:
        f2=f2+" "+wordnet_lemmatizer.lemmatize(w)
    doc = nlp(f2)

    tokens = [token.text for token in doc if not token.is_stop]

    l=[]
    l.append(tokens)
    pred.append(l)

print(pred)
df = pd.DataFrame(main, columns = ['Tokens', 'Label'])
df_pred = pd.DataFrame(pred, columns=['Tokens'])
cores = multiprocessing.cpu_count()
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5,
                     alpha=0.03,
                     min_alpha=0.0007,
                     negative=20,
                     workers=cores-1)

t = time()




[[['  ', 'EXECUTIVE', 'ASSISTANT', 'CHIEF', 'OPERATING', 'OFFICER', 'EXECUTIVE', 'ASSISTANT', 'CHIEF', 'OPERATING', 'OFFICER', 'SAI', 'CONSULTING', 'ENGINEERS', 'PVT', 'LTD', 'having', '18', 'year', 'experience', 'field', 'Office', 'management', 'administration', 'human', 'resource', 'management', 'secretarial', 'arena', 'Prior', 'joining', 'SAI', 'wa', 'associated', 'Intas', 'group', '8', 'year', 'carried', 'validation', 'HR', 'policy', 'implementation', 'good', 'HR', 'generalist', 'policy', 'good', 'administration', 'coordination', 'acumen', 'wa', 'instrumental', 'setting', 'office', 'cum', 'lab', 'Intas', 'group', 'company', 'Bavla', 'ha', 'experience', 'organizing', 'symposium', 'conference', 'National', 'repute', 'got', 'applause', 'successfully', 'organizing', 'event', 'form', 'Certificate', 'Intas', 'group', 'company', 'Work', 'Experience', 'EXECUTIVE', 'ASSISTANT', 'CHIEF', 'OPERATING', 'OFFICER', 'SAI', 'CONSULTING', 'ENGINEERS', 'PVT', 'LTD', 'Ahmedabad', 'Gujarat', 'June', '

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
w2v_model.build_vocab(df["Tokens"], progress_per=1000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

w2v_model.train(df["Tokens"], total_examples=w2v_model.corpus_count, epochs=10000, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.11 mins
Time to train the model: 1.4 mins


In [ ]:
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix

y = df['Label'].values
X = np.array(df["Tokens"])
X_pred = np.array(df_pred["Tokens"])

#And here is the train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)

matrix = vectorizer.fit_transform([x for x in X_train])

tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))

    count = 0.
    for word in tokens:
        try:

            vec += w2v_model[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count


    return vec

train_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_train)])


test_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_test)])



vocab size : 116


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:

train_vecs_w2v = scale(train_vecs_w2v)


test_vecs_w2v = scale(test_vecs_w2v)

pred_vec_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_pred)])
pred_vec_w2v = scale(pred_vec_w2v)


print ('shape for training set : ',train_vecs_w2v.shape,
      '\nshape for test set : ', test_vecs_w2v.shape,
       '\nshape for predicted set : ', pred_vec_w2v.shape,)

shape for training set :  (36, 300) 
shape for test set :  (9, 300) 
shape for predicted set :  (5, 300)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
import keras
from keras.models import Sequential, Model
from keras import layers
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Embedding
from keras.layers.merge import Concatenate

model = Sequential()

model.add(Dense(128, activation='relu', input_dim=300))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Using TensorFlow backend.






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               38528     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 38,657
Trainable params: 38,657
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(train_vecs_w2v, y_train, epochs=20, batch_size=50,
                   validation_data=(test_vecs_w2v,y_test))
loss, accuracy = model.evaluate(train_vecs_w2v, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))

loss, accuracy = model.evaluate(test_vecs_w2v, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


print(model.predict(pred_vec_w2v, verbose=False))




Train on 36 samples, validate on 9 samples
Epoch 1/20





36/36 [==============================] - 1s 15ms/step - loss: 0.8156 - acc: 0.5000 - val_loss: 0.7849 - val_acc: 0.7778
Epoch 2/20
36/36 [==============================] - 0s 248us/step - loss: 1.1860 - acc: 0.3611 - val_loss: 0.6448 - val_acc: 0.7778
Epoch 3/20
36/36 [==============================] - 0s 199us/step - loss: 0.5366 - acc: 0.7222 - val_loss: 0.5696 - val_acc: 0.7778
Epoch 4/20
36/36 [==============================] - 0s 184us/step - loss: 0.5351 - acc: 0.7778 - val_loss: 0.5014 - val_acc: 0.7778
Epoch 5/20
36/36 [==============================] - 0s 176us/step - loss: 0.4410 - acc: 0.8056 - val_loss: 0.4300 - val_acc: 0.8889
Epoch 6/20
36/36 [==============================] - 0s 186us/step - loss: 0.2517 - acc: 0.9167 - val_loss: 0.3986 - val_acc: 0.8889
Epoch 7/20
36/36 [==============================] - 0s 127us/step - loss: 0.3299 - acc: 0.8611 - val_loss: 0.3705 - val_acc: 0.8889
Epoch 8/20
36/36 [========

# New Section

# New Section

# New Section

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True